<a href="https://colab.research.google.com/github/ayush-kaurav0512/Homesite_quote_conversion/blob/main/Homesite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vecstack
#!pip install mlxtend
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from vecstack import stacking
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read training data and create a dataframe

train_df = pd.read_csv('/content/drive/MyDrive/machine_learning_dataset/RevisedHomesiteTrain1.csv')
train_df.head()

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,...,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
0,2,1,17,23,17,23,15,22,16,22,...,1,0,0,1,0,1,0,0,0,0
1,5,9,6,8,6,8,5,7,5,8,...,1,0,0,1,0,0,0,1,0,0
2,4,6,7,12,7,12,6,10,7,11,...,1,0,0,1,0,0,0,1,0,0
3,15,23,3,2,3,2,2,2,3,2,...,1,0,0,1,0,0,0,0,1,0
4,4,6,8,13,8,13,7,11,7,13,...,1,0,0,1,0,0,1,0,0,0


In [ ]:
# Check for null value
train_df.isnull().sum()

,0
CoverageField11A,0
CoverageField11B,0
CoverageField1A,0
CoverageField1B,0
CoverageField2A,0
...,...
GeographicField64_CA,0
GeographicField64_IL,0
GeographicField64_NJ,0
GeographicField64_TX,0


In [ ]:
# define cateogical and numerical cloumns

categorical_col=train_df.select_dtypes(include=['object']).columns
numerical_col=train_df.select_dtypes(include=['int64','float64']).columns

print(categorical_col)
print(numerical_col)

Index([], dtype='object')
Index(['CoverageField11A', 'CoverageField11B', 'CoverageField1A',
       'CoverageField1B', 'CoverageField2A', 'CoverageField2B',
       'CoverageField3A', 'CoverageField3B', 'CoverageField4A',
       'CoverageField4B',
       ...
       'PropertyField38_N', 'PropertyField38_Y', 'GeographicField63_ ',
       'GeographicField63_N', 'GeographicField63_Y', 'GeographicField64_CA',
       'GeographicField64_IL', 'GeographicField64_NJ', 'GeographicField64_TX',
       'QuoteConversion_Flag'],
      dtype='object', length=596)


In [ ]:
# Since encoding is already done check if data is balanced or not

train_df['QuoteConversion_Flag'].value_counts()

,count
QuoteConversion_Flag,
0,52738
1,12262


Since the data is clearly imbalanced, we have to perform re-sampling of data using Smote.

In [ ]:
# Balancing the train data

#define feature and target

X=train_df.drop(columns=['QuoteConversion_Flag'])
y=train_df['QuoteConversion_Flag']

# Apply smote

percentages=[0.3,0.4,0.5,0.7,0.8,0.9]
best_accuracy = 0
best_percentage = 0
accuracy=0

# Original smote
for percentage in percentages:
  sm=SMOTE(sampling_strategy=percentage, random_state=42)
  x_res,y_res=sm.fit_resample(X,y)

  X_train, X_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.2, random_state=42)

  # Train a Decision Tree model
  sm_model=DecisionTreeClassifier()
  sm_model.fit(X_train, y_train)

  # make a prediction
  y_pred=sm_model.predict(X_test)

  accuracy=accuracy_score(y_test, y_pred)

  # Evaluate the model and print the results
  print(f"SMOTE with {percentage*100:.0f}% oversampling:")
  print("Accuracy:", accuracy)
  print(classification_report(y_test, y_pred))
  print("-" * 30)

  if accuracy> best_accuracy:
    best_percentage = percentage

print(f"Best percentage: {best_percentage*100:.0f}%")
print(f"Accuracy for 90% oversampling: {accuracy}")

SMOTE with 30% oversampling:
Accuracy: 0.8838243873978997
              precision    recall  f1-score   support

           0       0.93      0.92      0.92     10573
           1       0.74      0.75      0.75      3139

    accuracy                           0.88     13712
   macro avg       0.84      0.84      0.84     13712
weighted avg       0.88      0.88      0.88     13712

------------------------------
SMOTE with 40% oversampling:
Accuracy: 0.8938172953206474
              precision    recall  f1-score   support

           0       0.93      0.92      0.93     10544
           1       0.81      0.82      0.82      4223

    accuracy                           0.89     14767
   macro avg       0.87      0.87      0.87     14767
weighted avg       0.89      0.89      0.89     14767

------------------------------
SMOTE with 50% oversampling:
Accuracy: 0.8957780305903172
              precision    recall  f1-score   support

           0       0.93      0.92      0.92     10645
 

Since Smote with 90% oversampling has the most accuracy will be used.

In [ ]:
# Now apply smote with best percentage

sm=SMOTE(sampling_strategy=best_percentage, random_state=42)
x_res,y_res=sm.fit_resample(X,y)

#Apply test-train split

X_train, X_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.2, random_state=42)



In [ ]:
# Train DecisonTreeModel
dt_model=DecisionTreeClassifier()
#dt_model.fit(X_train, y_train)

In [ ]:
# Random forest
rf_model=RandomForestClassifier()
#rf_model.fit(X_train, y_train)

In [ ]:
# Linear Support vector machine
lsvm_model=LinearSVC()
# lsvm_model.fit(X_train, y_train)

In [ ]:
# Multilayer Perceptron
mlp_model=MLPClassifier()
# mlp_model.fit(X_train, y_train)

In [ ]:
# K-nearest neighbors
knn_model=KNeighborsClassifier()
# knn_model.fit(X_train, y_train)

In [ ]:
# Read test data

test_df = pd.read_csv('/content/drive/MyDrive/machine_learning_dataset/RevisedHomesiteTest1.csv')

test_df=test_df.drop(columns=['GeographicField64'])

In [ ]:
# check for null values
test_df.isnull().sum()

,0
CoverageField11A,0
CoverageField11B,0
CoverageField1A,0
CoverageField1B,0
CoverageField2A,0
...,...
GeographicField63_Y,0
GeographicField64_CA,0
GeographicField64_IL,0
GeographicField64_NJ,0


In [ ]:
numerical_col=test_df.select_dtypes(include=['int64','float64']).columns
categorical_col=test_df.select_dtypes(include=['object']).columns

print(test_df.dtypes)

print(test_df.head())

pred_test=test_df

CoverageField11A        int64
CoverageField11B        int64
CoverageField1A         int64
CoverageField1B         int64
CoverageField2A         int64
                        ...  
GeographicField63_Y     int64
GeographicField64_CA    int64
GeographicField64_IL    int64
GeographicField64_NJ    int64
GeographicField64_TX    int64
Length: 595, dtype: object
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                13                22                4                4   
1                 4                 5                8               14   
2                 3                 3               11               18   
3                 5                 9               14               22   
4                12                21                4                5   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0                4                4                3                3   
1                8               14           

In [ ]:
# Ensemble Stacking and stacked model

models=[dt_model, rf_model, lsvm_model, mlp_model, knn_model]

S_train, S_test= stacking(
    models,x_res, y_res,pred_test,
    regression=False,
    mode='oof_pred_bag',
    needs_proba=False,
    save_dir=None,
    metric=accuracy_score,
    n_folds=4,
    stratified=True,
    shuffle=True,
    random_state=0,
    verbose=2)


task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.91700930]
    fold  1:  [0.91744841]
    fold  2:  [0.91473054]
    fold  3:  [0.91968064]
    ----
    MEAN:     [0.91721722] + [0.00175704]
    FULL:     [0.91721722]

model  1:     [RandomForestClassifier]
    fold  0:  [0.93860525]
    fold  1:  [0.93840565]
    fold  2:  [0.93632735]
    fold  3:  [0.93588822]
    ----
    MEAN:     [0.93730662] + [0.00121090]
    FULL:     [0.93730664]

model  2:     [LinearSVC]
    fold  0:  [0.92335635]
    fold  1:  [0.93920402]
    fold  2:  [0.92123752]
    fold  3:  [0.92966068]
    ----
    MEAN:     [0.92836465] + [0.00698303]
    FULL:     [0.92836470]

model  3:     [MLPClassifier]
    fold  0:  [0.65226937]
    fold  1:  [0.91281785]
    fold  2:  [0.89005988]
    fold  3:  [0.76083832]
    ----
    MEAN:     [0.80399636] + [0.10503897]
    FULL:     [0.8

In [ ]:
# best stack model
tree_parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
mlp_parameters = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
}
knn_parameters = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Create a stack model

stack_model={
    "KNeighborsClassifier": KNeighborsClassifier(),
    "MLPClassifier": MLPClassifier()
    "RandomForestClassifier": RandomForestClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
    }
for model_name, model in stack_model.items():
      print(f"Training {model_name}...")
      print(model)

      # different parameter
      if model_name == "KNeighborsClassifier":
          parameters = knn_parameters
      elif model_name == "MLPClassifier":
          parameters = mlp_parameters
      else:
          parameters = tree_parameters

      # fit the model
      model.fit(S_train, y_res)

      # Hypertuning using RandomSearch
      if parameters:
        stack_random = RandomizedSearchCV(model,parameters,n_iter=15)
        stack_random.fit(S_train, y_res)
        stck_best_param=stack_random.best_params_
        final_stack_model=model.__class__(**stck_best_param)
      else:
        final_stack_model=model

      final_stack_model.fit(S_train, y_res)
      Y_pred=final_stack_model.predict(S_test)

      # create a data frame to store Y_pred

      pred_df=pd.DataFrame(Y_pred, columns=['QuoteConversion_Flag'])

      # concat the data frame
      # final_pred_df=pd.concat([test_df['QuoteNumber'], pred_df], axis=1)
      # final_pred_df.to_csv(f"{model_name}_submission.csv", index=False)
      # final_stack_model.fit(S_train, y_res)
      # Y_pred=final_stack_model.predict(S_test)

      # # create a data frame to store Y_pred

      # pred_df=pd.DataFrame(Y_pred, columns=['QuoteConversion_Flag'])

      # concat the data frame
      final_pred_df=pd.concat([test_df['QuoteNumber'], pred_df], axis=1)
      final_pred_df.to_csv(f"{model_name}_submission.csv", index=False)

Training KNeighborsClassifier...
KNeighborsClassifier()
Training MLPClassifier...
MLPClassifier()
